# Customer Churn Data Preparation

In [72]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
import time

In [ ]:
user = "root"
password = "7003890541"
host = "localhost"
port = "3306"
database = "churn_final_project" 

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

In [12]:
query = ''' select * from backup_churn limit 5'''
pd.read_sql(query,engine)

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [53]:
def sql(query=query,engine = engine):
    return pd.read_sql(query,engine)

In [30]:
query = '''
select
    (select count(*) from backup_churn) as row_count,
    (select count(*) from information_schema.columns
            where table_name = 'backup_churn'
            and table_schema = database()) as column_count
'''
sql(query=query)

,row_count,column_count
0,440833,12


`Being a large dataset, I will use mysql for cleaning and manipulating data`

In [26]:
query = '''
    create table if not exists data as
        select * from backup_churn;
'''
with engine.connect() as conn:
    conn.execute(text(query))

**New data created so that original data is not changed accidentally**

In [31]:
query = '''
    select * from data limit 5;
'''
sql(query=query)

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [34]:
# Description of data
query = ''' describe data;'''
sql(query).iloc[:,:2]

,Field,Type
0,CustomerID,double
1,Age,double
2,Gender,text
3,Tenure,double
4,Usage Frequency,double
5,Support Calls,double
6,Payment Delay,double
7,Subscription Type,text
8,Contract Length,text
9,Total Spend,double


In [36]:
# Renaming columns
query ='''
    alter table data
    rename column CustomerID to id,
    rename column Age to age,
    rename column Gender to gender,
    rename column Tenure to tenure,
    rename column `Usage Frequency` to frequency,
    rename column `Support Calls` to calls,
    rename column `Payment Delay` to payment_delay,
    rename column `Subscription Type` to subscription,
    rename column `Contract Length` to contract,
    rename column `Total Spend` to spend,
    rename column `Last Interaction` to last_interaction,
    rename column Churn to churn;

'''
with engine.connect() as conn:
    conn.execute(text(query))


In [38]:
query = 'describe data;'
sql(query).iloc[:,:2]

,Field,Type
0,id,double
1,age,double
2,gender,text
3,tenure,double
4,frequency,double
5,calls,double
6,payment_delay,double
7,subscription,text
8,contract,text
9,spend,double


**Extracting numerical and categorical columns for easy processing**

In [41]:
num_cols = ['age','tenure','frequency','calls','payment_delay','spend','last_interaction']
cat_cols = ['gender','subscription','contract','churn']
all_cols = num_cols+cat_cols

In [97]:
for col in all_cols:
    query = f''' 
        select '{col}' as col, sum({col} is null) as null_count from data;
    '''
    print(sql(query))


   col  null_count
0  age         0.0
      col  null_count
0  tenure         0.0
         col  null_count
0  frequency         0.0
     col  null_count
0  calls         0.0
             col  null_count
0  payment_delay         0.0
     col  null_count
0  spend         0.0
                col  null_count
0  last_interaction         0.0
      col  null_count
0  gender         0.0
            col  null_count
0  subscription         0.0
        col  null_count
0  contract         0.0
     col  null_count
0  churn         0.0


In [98]:
query = '''
    select * from data
    where id is null or
    age is null or
    gender is null or
    tenure is null or
    frequency is null or
    calls is null or 
    payment_delay is null or
    subscription is null or
    contract is null or
    spend is null or
    last_interaction is null or
    churn is null;
'''
sql()

,id,age,gender,tenure,frequency,calls,payment_delay,subscription,contract,spend,last_interaction,churn


In [115]:
for col in all_cols:
    query = f'''
        select '{col}' as col,
        count(distinct {col}) as dist from data
    '''
    print(sql(query))


   col  dist
0  age    48
      col  dist
0  tenure    60
         col  dist
0  frequency    30
     col  dist
0  calls    11
             col  dist
0  payment_delay    31
     col   dist
0  spend  68363
                col  dist
0  last_interaction    30
      col  dist
0  gender     2
            col  dist
0  subscription     3
        col  dist
0  contract     3
     col  dist
0  churn     2


In [121]:
for col in cat_cols:
    query = f'''
        select distinct {col} from data;
    '''
    print(sql(query))

   gender
0  Female
1    Male
  subscription
0     Standard
1        Basic
2      Premium
    contract
0     Annual
1    Monthly
2  Quarterly
   churn
0    1.0
1    0.0


In [124]:
query = '''
    update data
    set gender = lower(gender),
    subscription = lower(subscription),
    contract = lower(contract);
'''

with engine.begin() as con:
    con.execute(text(query))

In [127]:
for col in cat_cols:
    query = f'''
        select distinct {col} from data;
    '''
    print(sql(query))

   gender
0  female
1    male
  subscription
0     standard
1        basic
2      premium
    contract
0     annual
1    monthly
2  quarterly
   churn
0    1.0
1    0.0


`There is no categorical column with wrong values`

In [130]:
query = f'''
    select {",".join(num_cols)} from data;
    '''
sql(query).describe().round(2)

,age,tenure,frequency,calls,payment_delay,spend,last_interaction
count,440832.00,440832.00,440832.00,440832.00,440832.00,440832.00,440832.00
mean,39.37,31.26,15.81,3.60,12.97,631.62,14.48
std,12.44,17.26,8.59,3.07,8.26,240.80,8.60
min,18.00,1.00,1.00,0.00,0.00,100.00,1.00
25%,29.00,16.00,9.00,1.00,6.00,480.00,7.00
50%,39.00,32.00,16.00,3.00,12.00,661.00,14.00
75%,48.00,46.00,23.00,6.00,19.00,830.00,22.00
max,65.00,60.00,30.00,10.00,30.00,1000.00,30.00


`There is no numerical column with invalid values`

In [132]:
query = '''
    select * from data limit 5
'''
sql(query)

,id,age,gender,tenure,frequency,calls,payment_delay,subscription,contract,spend,last_interaction,churn
0,2.0,30.0,female,39.0,14.0,5.0,18.0,standard,annual,932.0,17.0,1.0
1,3.0,65.0,female,49.0,1.0,10.0,8.0,basic,monthly,557.0,6.0,1.0
2,4.0,55.0,female,14.0,4.0,6.0,18.0,basic,quarterly,185.0,3.0,1.0
3,5.0,58.0,male,38.0,21.0,7.0,7.0,standard,monthly,396.0,29.0,1.0
4,6.0,23.0,male,32.0,20.0,5.0,8.0,basic,monthly,617.0,20.0,1.0
